# Solving Sudoku using Propositional Logic

## Project Overview
This project implements a Sudoku solver using propositional logic and SAT solving techniques from AIMA's logic module. While initially considering both propositional and first-order logic approaches, we ultimately chose propositional logic due to limitations in AIMA's FOL implementation regarding constraint representation.

## Implementation Approach

### Why Propositional Logic?
We represent each Sudoku cell as a propositional variable in the form `RiCjNk`, where:
- `i` = row number (1-9)
- `j` = column number (1-9)
- `k` = digit (1-9)

For example, `R1C3N7` means "the cell at row 1, column 3 contains the number 7".

### Key Components
1. **Knowledge Base**: Uses PropKB to store and manage logical sentences
2. **Constraints**: Implements four types of Sudoku rules:
   - Row constraints: Each number appears once in each row
   - Column constraints: Each number appears once in each column
   - Box constraints: Each number appears once in each 3x3 box
   - Cell constraints: Each cell contains exactly one number
3. **Solver**: Uses DPLL (Davis–Putnam–Logemann–Loveland) algorithm for SAT solving

## Why Not First-Order Logic?

### AIMA's FOL Module Limitations
While First-Order Logic (FOL) might seem like a natural choice for Sudoku due to its expressive power, we encountered fundamental limitations in AIMA's FOL implementation:

### Definite Clause Restriction
AIMA's FOL inference functions (`fol_fc_ask()` and `fol_bc_ask()`) only work with definite clauses:

✅ Valid definite clauses:
```text
Human(Socrates)                              # A fact
Mortal(x) :- Human(x)                        # Single positive conclusion
Grandparent(x,z) :- Parent(x,y) ∧ Parent(y,z) # Rule with conjunctive body
```

❌ Invalid clauses (needed for Sudoku):
```text
~Filled(1,1,2) ∨ ~Filled(1,1,3)  # Constraint with only negative literals
Filled(1,1,x) ∨ Filled(1,1,y)     # Multiple positive literals
```

### Impact on Sudoku Implementation
Sudoku's core constraints often involve mutual exclusions (e.g., "a cell cannot contain both 2 and 3"), which translate to clauses with only negative literals. These cannot be expressed as definite clauses, making FOL implementation impractical with AIMA's current limitations.

This limitation led us to choose propositional logic with SAT solving, which perfectly handles such constraints and provides an efficient solution mechanism.

In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(parent_dir, 'aima'))

from logic import *

## Implementation Details

### Class Structure
The solver is implemented in the `SudokuSolverPropKB` class with these key methods:

1. `__init__(board)`: Initializes solver with optional board input
2. `_sentence(r, c, n)`: Creates propositional variables in RiCjNk format
3. `_addconstraints()`: Adds all Sudoku rules to the knowledge base
4. `solve()`: Uses DPLL to find a solution

### Dependencies
We use AIMA's logic module for propositional logic functionality:

In [2]:
class SudokuSolverPropKB:
    def __init__(self, board: list[list[int]]=None) -> None:
        ''' Initialize the SudokuSolverPropKB with a given board

        Args:
            board (list[list[int]], optional): A 9x9 Sudoku board. Defaults to None.
        '''
        self.kb = PropKB()
        self.board = board
        self._addconstraints()

    def _sentence(self, r: int, c: int, n: int) -> str:
        ''' Define a sentence in the form of R{r}C{c}N{n}

        Args:
            r (int): Row number
            c (int): Column number
            n (int): Number in the cell

        Returns:
            str: A string representing the cell in the form of R{r}C{c}N{n}
        '''
        return f'R{r}C{c}N{n}'

    def _addconstraints(self) -> None:
        ''' Add constraints to the knowledge base
        '''
        self._add_row_constraints()
        self._add_column_constraints()
        self._add_box_constraints()
        self._add_cell_constraints()
        self._add_known()

    def _add_row_constraints(self) -> None:
        ''' Add row constraints to the knowledge base
        '''
        for r in range(9):
            for n in range(1, 10):
                for c1 in range(9):
                    for c2 in range(c1 + 1, 9):
                        self.kb.tell(expr(f'~{self._sentence(r, c1, n)} | ~{self._sentence(r, c2, n)}'))

    def _add_column_constraints(self) -> None:
        ''' Add column constraints to the knowledge base
        '''
        for c in range(9):
            for n in range(1, 10):
                for r1 in range(9):
                    for r2 in range(r1 + 1, 9):
                        self.kb.tell(expr(f'~{self._sentence(r1, c, n)} | ~{self._sentence(r2, c, n)}'))

    def _add_box_constraints(self) -> None:
        ''' Add square constraints to the knowledge base
        '''
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                for n in range(1, 10):
                    cells = [
                        (r, c)
                        for r in range(i, i + 3)
                        for c in range(j, j + 3)
                    ]
                    for idx, (r1, c1) in enumerate(cells):
                        for r2, c2 in cells[idx + 1:]:
                            self.kb.tell(expr(f'~{self._sentence(r1, c1, n)} | ~{self._sentence(r2, c2, n)}'))

    def _add_cell_constraints(self) -> None:
        ''' Add cell constraints to the knowledge base
        '''
        for r in range(9):
            for c in range(9):
                sentences = []
                for n in range(1, 10):
                    sentences.append(self._sentence(r, c, n))
                self.kb.tell(expr(' | '.join(sentences)))

        for r in range(9):
            for c in range(9):
                for n1 in range(1, 10):
                    for n2 in range(n1 + 1, 10):
                        self.kb.tell(expr(f'~{self._sentence(r, c, n1)} | ~{self._sentence(r, c, n2)}'))

    def _add_known(self) -> None:
        ''' Add known values to the knowledge base
        '''
        if self.board:
            for r in range(9):
                for c in range(9):
                    if self.board[r][c] != 0:
                        self.kb.tell(expr(self._sentence(r, c, self.board[r][c])))

    def solve(self) -> list[list[int]]:
        ''' Solve the Sudoku puzzle

        Returns:
            list[list[int]]: A 9x9 Sudoku board with the solution
        '''
        self.board = self.board or [[0] * 9 for _ in range(9)]
        model = dpll_satisfiable(associate('&',self.kb.clauses))

        if model:
          for r in range(9):
              for c in range(9):
                  if self.board[r][c] == 0:
                      for n in range(1, 10):
                          if model.get(expr(self._sentence(r, c, n)), False):
                              self.board[r][c] = n
                              break
          return self.board

        return None

In [3]:
board = [
    [0, 1, 0, 0, 4, 0, 0, 5, 0],
    [4, 0, 7, 0, 0, 0, 6, 0, 2],
    [8, 2, 0, 6, 0, 0, 0, 7, 4],
    [0, 0, 0, 0, 1, 0, 5, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 4, 0, 5, 0, 0, 0, 0],
    [9, 6, 0, 0, 0, 3, 0, 4, 5],
    [3, 0, 5, 0, 0, 0, 8, 0, 1],
    [0, 7, 0, 0, 2, 0, 0, 3, 0],
]

## Usage Example

Below we demonstrate solving a sample Sudoku puzzle. The input format is:
- 9x9 list of lists
- 0 represents empty cells
- 1-9 represent filled cells

In [5]:
solver = SudokuSolverPropKB(board)

In [ ]:
solution = solver.solve()

In [6]:
solution

[[6, 1, 9, 7, 4, 2, 3, 5, 8],
 [4, 5, 7, 8, 3, 1, 6, 9, 2],
 [8, 2, 3, 6, 9, 5, 1, 7, 4],
 [2, 3, 6, 4, 1, 9, 5, 8, 7],
 [5, 9, 1, 2, 7, 8, 4, 6, 3],
 [7, 8, 4, 3, 5, 6, 2, 1, 9],
 [9, 6, 2, 1, 8, 3, 7, 4, 5],
 [3, 4, 5, 9, 6, 7, 8, 2, 1],
 [1, 7, 8, 5, 2, 4, 9, 3, 6]]

## Results and Performance

The solver successfully finds solutions to Sudoku puzzles using the DPLL algorithm. The propositional logic approach has several advantages:

1. **Complete**: Guaranteed to find a solution if one exists
2. **Sound**: All solutions are valid according to Sudoku rules
3. **Efficient**: SAT solving with DPLL performs well for typical puzzles

### Logical Complexity
- Number of variables: 9 x 9 x 9 = 729 (each cell can contain any digit)
- Constraints: Row, column, box, and cell constraints converted to CNF

### Future Improvements
1. Add visualization of the solving process
2. Implement puzzle generation
3. Add difficulty rating system
4. Optimize constraint generation